## 4.1 임베딩이란? 

임베딩은 단어나 문장을 수치화해 벡터 공간으로 표현하는 과정을 의미한다. 따라서 딥러닝 모델의 입력값으로 많이 사용된다. 말뭉치의 의미에 따라 벡터화되기 때문에 문법적인 정보가 포함되어 있다. 이 때문에 임베딩 품질이 좋다면 간단한 모델로도 높은 성능을 보일 수 있다.

**<임베딩 기법>**

* 문장 임베딩 : 문장 전체를 벡터화하는 방법, 문맥적 의미를 내포할 수 있다. 하지만 많은 문장 데이터가 필요하고, 학습 시 비용이 많이 들어간다.
* 단어 임베딩 : 개별 단어를 벡터로 표현하는 방법, 문장 임베딩에 비해 학습 방법이 간단해 실무에서 여전히 많이 사용된다. 그러나 동음이의어에 대한 구분을 하지 않아 의미가 다르더라도 단어의 형태가 같다면 동일한 벡터값으로 표현된다.



## 4.2 단어 임베딩

### 4.2.1 원-핫 인코딩

단어를 숫자 벡터로 변환하는 가장 기본적인 방법이다. 이름에서 알 수 있듯 요소들 중 하나의 값만 1을 가지고 나머지는 0을 갖는 인코딩을 의미한다. 전체 요소 중 단 하나의 값만 1이기에 희소(sparse) 벡터라고 한다.

In [ ]:
!pip install konlpy

In [4]:
from konlpy.tag import Komoran
import numpy as np

komoran = Komoran()
text = "오늘 하루는 정말 피곤하네요."

nouns = komoran.nouns(text)
print(nouns)

dics = {}
for word in nouns:
  if word not in dics.keys():
    dics[word] = len(dics)
print(dics)

# 원-핫 인코딩!
nb_classes = len(dics)
targets = list(dics.values())
one_hot_targets = np.eye(nb_classes)[targets] # np.eye : 단위행렬을 만들어주는 함수
print(one_hot_targets)

['오늘', '하루', '피곤']
{'오늘': 0, '하루': 1, '피곤': 2}
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


### 4.2.2 희소 표현과 분산 표현 

희소 표현은 단어 사전의 길이가 길어지면 길어질 수록 메모리 낭비와 계산 복잡도가 커지는 단점이 있다. 또한 단어 간의 연관성이 전혀 없어 의미를 담을 수 없다. 또한 차원의 저주에 걸릴 가능성이 매우 높아진다. 이러한 문제를 해결하기 위해 고안된 방법은 분산 표현(distributed representation)이라고 한다. 분산 표현은 한 단어의 정보가 특정 차원에 표현되지 않고 여러 차원에 분산되어 표현된다 하여 지어진 이름이다.
   
신경망이 분산 표현을 학습하는 과정에서 임베딩 벡터의 모든 차원에 의미있는 데이터를 고르게 밀집시킨다. 이 때문에 데이터 손실을 최소화하면서 벡터 차원이 압축되는 효과가 생긴다. 분산 표현 방식은 밀집 표현(dense representation)이라 부르기도 하면서, 밀집 표현으로 만들어진 벡터를 밀집 벡터(dense vector)라고 한다.

**<분산 표현의 장점>**
* 임베딩 벡터의 차원을 데이터 손실을 최소화하며 압축할 수 있다. 입력 데이터의 차원이 너무 높아질 경우, 신경망 모델의 학습이 어려워지는 차원의 저주 문제가 발생한다.
* 임베딩 벡터에는 단어의 의미, 주변 단어간의 관계 등 많은 정보가 내포되어 있어 일반화 능력이 뛰어나다.

### 4.2.3 Word2Vec

챗봇의 경우 많은 단어를 처리하면서 단어 간 유사도를 계산할 수 있어야 좋은 성능을 낼 수 있기 때문에 원-핫 인코딩 기법은 좋은 선택이 아니다. 대표적인 신경망 단어 임베딩 방법인 Word2Vec을 소개하고 사용방법을 알아 보겠다. 

Word2Vec 모델은 CBOW, skip-gram 총 두가지 모델로 제안되었다. CBOW는 맥락(context word)라 불리는 주변 단어들을 이용해 타깃 단어를 예측하는 신경망 모델이다. 신경망의 입력을 주변 단어들로 구성하고 출력을 타깃 단어로 설정해학습된 가중치 데이터를 임베딩 벡터로 활용한다. 타깃 단어를 예측하기 위해 앞 뒤로 몇 개의 단어를 확인할지 결정할 수 있는데, 이 범위를 윈도우(window)라고 한다.

이와 반대로 skip-gram 모델은 하나의 타깃 단어를 이용해 주변 단어들을 예측하는 신경망 모델이다. skip-gram이 CBOW에 비해 예측해야 하는 맥락이 많아 단어 분산 표현력이 우수해 임베딩 품질이 우수하다.

In [5]:
import urllib
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")

('ratings_train.txt', <http.client.HTTPMessage at 0x7f0c039eea20>)

In [6]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran

# 네이버 영화 리뷰 데이터를 사용한다
def read_review_data(filename):
  with open(filename, 'r') as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:]
  return data

print('1) 말뭉치 데이터 읽기 시작')
review_data = read_review_data('/content/ratings_train.txt')
print(len(review_data))

print('2) 형태소에서 명사 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data]

print('3) 모델 학습 시작')
# sentences : 모델 학습에 필요한 데이터, size : 단어 임베딩 벡터 차원, window : 윈도우 크기
# hs : 0(0이 아닌 경우 음수 샘플링 사용), 1(모델 학습에 softmax 사용), min_count : 단어 최소 빈도수 제한, sg : 0(CBOW), 1(skip_gram)
model = Word2Vec(sentences = docs, size = 200, window = 4, hs = 1, min_count = 2, sg = 1)

print('4) 학습된 모델 저장 시작')
model.save('nvmc.model')

print('corpus_count : ', model.corpus_count)
print('corpus_total_words : ', model.corpus_total_words)

1) 말뭉치 데이터 읽기 시작
150000
2) 형태소에서 명사 추출 시작
3) 모델 학습 시작
4) 학습된 모델 저장 시작
corpus_count :  150000
corpus_total_words :  806264


**만들어진 모델 읽어와 실제로 단어 임베딘된 값과 벡터 공간상의 유사한 단어들을 확인**

In [7]:
from gensim.models import Word2Vec

model = Word2Vec.load('/content/nvmc.model')
print('corpus_total_words : ', model.corpus_total_words)

# '사랑'이라는 단어로 생성한 임베딩 벡터
print('사랑 ; ', model.wv['사랑'])

# 단어 유사도 계산
print('일요일 = 월요일\t', model.wv.similarity(w1 = '일요일', w2 = '월요일'))

# 가장 유사한 단어 추출
print(model.wv.most_similar('안성기', topn = 5))

corpus_total_words :  806264
사랑 ;  [-6.74988031e-02  6.00074530e-01 -7.62046650e-02  1.71618193e-01
  6.58940449e-02 -8.51436406e-02 -1.42345220e-01  1.20324470e-01
  4.26264144e-02 -3.90026659e-01 -8.63191579e-03 -1.17552891e-01
  3.94470729e-02  1.78302869e-01 -9.89935473e-02  3.65931541e-02
  3.63852948e-01 -2.61451825e-02 -2.89338231e-02  3.69054675e-02
 -3.21664006e-01 -1.01783007e-01  1.64149269e-01  1.14270300e-01
 -1.71412736e-01 -1.62363142e-01  1.15882218e-01  9.93937626e-02
  1.31306961e-01  4.28856127e-02 -1.88654080e-01 -1.49762660e-01
  3.61318231e-01 -1.10992528e-01  2.32460633e-01 -1.61209926e-01
 -8.91700089e-02  3.06289166e-01  7.72543252e-02 -1.90806195e-01
  2.04794779e-01  2.32813329e-01 -3.26476455e-01  2.61789709e-01
  2.06414431e-01  1.18425757e-01  3.20243448e-01 -1.01320200e-01
 -2.83871889e-01  3.31708521e-01 -1.05258331e-01 -2.26199880e-01
 -5.02921753e-02  3.78054142e-01  3.43502387e-02  5.07646836e-02
  1.66489952e-03  4.20453101e-01 -1.66918606e-01  9.024